#ResNet Pythorch MNIST 
cf.https://www.kaggle.com/readilen/resnet-for-mnist-with-pytorch


In [0]:
#ライブラリのインポート
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
data_folder = '~/data'
BATCH_SIZE = 8

#学習データ
train_data_with_labels = MNIST(data_folder, train=True, download=True, transform=transforms.ToTensor())
train_data_loader = DataLoader(train_data_with_labels, batch_size=BATCH_SIZE, shuffle=True)

#検証データ
test_data_with_labels = MNIST(data_folder, train=False, download=True, transform=transforms.ToTensor())
test_data_loader = DataLoader(test_data_with_labels, batch_size=BATCH_SIZE, shuffle=True)

0it [00:00, ?it/s]

9920512it [00:01, 8455000.19it/s]                            


Extracting /root/data/MNIST/raw/train-images-idx3-ubyte.gz to /root/data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 116458.10it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/data/MNIST/raw


1654784it [00:00, 2108635.05it/s]                            
0it [00:00, ?it/s]

Extracting /root/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/data/MNIST/raw


8192it [00:00, 46651.15it/s]            


Extracting /root/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/data/MNIST/raw
Processing...
Done!


In [0]:
# 3*3 convolutino
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                    stride=stride, padding=1, bias=False)


# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(1, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [0]:
net_args = {
    "block": ResidualBlock,
    "layers": [2, 2, 2, 2]
}
model = ResNet(**net_args)

# Cross Entropy Loss 
lossResult = nn.CrossEntropyLoss()

# Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [10]:
MAX_EPOCH = 4

for epoch in range(MAX_EPOCH):
  total_loss = 0.0
  for i, data in enumerate(train_data_loader):
    #dataから学習対象データと教師ラベルデータを取り出す
    train_data, teacher_labels = data

    #入力をtorch.autograd.Variableに変換
    #train_data = Variable(train_data)
    train_data = Variable(train_data.resize_(BATCH_SIZE, 1, 32, 32))
    teacher_labels = Variable(teacher_labels)

    #計算された勾配情報を削除
    optimizer.zero_grad()

    #モデルに学習データを与えて予測を計算
    outputs = model(train_data)

    #lossとwによる微分計算
    loss = lossResult(outputs, teacher_labels)
    loss.backward()

    #勾配を更新
    optimizer.step()

    #誤差を計算する
    total_loss += loss.detach().numpy()

    #2000ミニバッチずつ、進捗を表示する
    if i % 2000 == 1999:
      print('学習進捗：[%d, %d] 学習誤差（loss）: %.3f' % (epoch+1, i+1, 100*total_loss/2000))
      total_loss = 0.0
  
  print('学習終了')

学習進捗：[1, 2000] 学習誤差（loss）: nan
学習進捗：[1, 4000] 学習誤差（loss）: nan
学習進捗：[1, 6000] 学習誤差（loss）: nan
学習終了
学習進捗：[2, 2000] 学習誤差（loss）: nan
学習進捗：[2, 4000] 学習誤差（loss）: nan
学習進捗：[2, 6000] 学習誤差（loss）: nan
学習終了
学習進捗：[3, 2000] 学習誤差（loss）: nan
学習進捗：[3, 4000] 学習誤差（loss）: nan
学習進捗：[3, 6000] 学習誤差（loss）: nan
学習終了
学習進捗：[4, 2000] 学習誤差（loss）: nan
学習進捗：[4, 4000] 学習誤差（loss）: nan
学習進捗：[4, 6000] 学習誤差（loss）: nan
学習終了


学習進捗：[1, 2000] 学習誤差（loss）: nan<br>
学習進捗：[1, 4000] 学習誤差（loss）: nan<br>
学習進捗：[1, 6000] 学習誤差（loss）: nan<br>
学習終了<br>
学習進捗：[2, 2000] 学習誤差（loss）: nan<br>
学習進捗：[2, 4000] 学習誤差（loss）: nan<br>
学習進捗：[2, 6000] 学習誤差（loss）: nan<br>
学習終了<br>
学習進捗：[3, 2000] 学習誤差（loss）: nan<br>
学習進捗：[3, 4000] 学習誤差（loss）: nan<br>
学習進捗：[3, 6000] 学習誤差（loss）: nan<br>
学習終了<br>
学習進捗：[4, 2000] 学習誤差（loss）: nan<br>
学習進捗：[4, 4000] 学習誤差（loss）: nan<br>
学習進捗：[4, 6000] 学習誤差（loss）: nan<br>
学習終了

In [36]:
#トータル
total = 0
#正解カウンター
count_when_correct = 0
iiii = 0

for data in test_data_loader:
  #検証ローダーからデータを取り出した上、アンパック
  test_data, teacher_labels = data
  #テストデータを変換した上、モデルに渡して、判定
  results = model(Variable(test_data.resize_(BATCH_SIZE, 1, 32, 32)))
#  results = model(Variable(test_data.resize_(1,1,32,32)))
  #予測を取り出す
  _, predicted = torch.max(results.data, 1)
  #
  total += teacher_labels.size(0)
  count_when_correct += (predicted == teacher_labels).sum()

print('count_when_correct:%d' % (count_when_correct))
print('total:%d' % (total))

print('正解率:%d / %d = %f' % (count_when_correct, total, int(count_when_correct) / int(total)))

count_when_correct:980
total:10000
正解率:980 / 10000 = 0.098000


count_when_correct:980<br>
total:10000<br>
正解率:980 / 10000 = 0.098000<br>